In [19]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy
from arcgis.features import FeatureLayer
import re

# Make Feature Layer

In [20]:
sdfCombinedForecasts = pd.DataFrame.spatial.from_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp')

In [21]:
sdfCombinedForecasts = sdfCombinedForecasts[sdfCombinedForecasts['ROUTE']!='UTA']
sdfCombinedForecasts

,FID,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,F2019,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0007_000.0,0007,0.0,0.076,0.0808,53,3,Dixie,3032,...,1.0082,1.033,1.0393,1.0366,0.9847,0.9221,4,1.071,0,"{""paths"": [[[268173.08999999985, 4101180.67], ..."
1,1,0007_000.1,0007,0.076,0.297,0.225062,53,3,Dixie,3032,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.051,10,1.1264,0,"{""paths"": [[[268284.90490000043, 4101114.28910..."
2,2,0007_000.3,0007,0.297,1.525,1.233974,53,3,Dixie,4357,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.051,10,1.1264,0,"{""paths"": [[[268529.2000000002, 4100859.150000..."
3,3,0007_001.5,0007,1.525,2.399,0.875242,53,3,Dixie,4357,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.051,10,1.1264,0,"{""paths"": [[[269520.8300000001, 4099147.59], [..."
4,4,0007_002.4,0007,2.399,3.449,1.0519,53,3,Dixie,4357,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.051,10,1.1264,0,"{""paths"": [[[270539.79000000004, 4098210], [27..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8932,8932,UDOT_7029,UDOT,0.0,0.0,0.238632,45,0,UDOT,0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0,"{""paths"": [[[387407.20990000013, 4501866.63780..."
8933,8933,UDOT_7035,UDOT,0.0,0.0,0.56169,45,0,UDOT,-2000,...,1.0085,1.032,1.0208,1.0204,0.9902,0.9587,8,1.0414,0,"{""paths"": [[[386817.73319999967, 4501197.0429]..."
8934,8934,UDOT_7040,UDOT,0.0,0.0,0.073807,29,0,UDOT,0,...,1.01,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0,"{""paths"": [[[444051.7000000002, 4543405.9], [4..."
8935,8935,UDOT_7060,UDOT,0.0,0.0,2.839235,45,0,UDOT,0,...,1.0085,1.032,1.0208,1.0204,0.9902,0.9587,8,1.0414,0,"{""paths"": [[[387328.7209000001, 4488243.8895],..."


In [22]:
#already done in previous notebook
#sdfCombinedForecasts.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

# Create Route Json

In [23]:
# Route Json
# R: Route Label (string, first 4 of SEGID)
# C: CO_FIPS (integer)
# label: Prefix of 
#        "I-"  + Route Label with 0s removed : for interstates
#        "Hwy" + Route Label with 0s removed : us and state routes
#        "FA"  + Route Label with 0s removed : federal aid routes
#        "WFRC/MAG/UDOT/UTA" 


dfRoutePrefix = pd.DataFrame([
    [   1,   14, 'Hwy'],
    [  15,   15, 'I-' ],
    [  16,   69, 'Hwy'],
    [  70,   70, 'I-' ],
    [  71,   79, 'Hwy'],
    [  80,   80, 'I-' ],
    [  81,   83, 'Hwy'],
    [  84,   84, 'I-' ],
    [  85,  214, 'Hwy'],
    [ 215,  215, 'I-' ],
    [ 216,  999, 'Hwy'],
    [1000, 5000, 'FA' ]
], columns=('routeFrom','routeTo','prefix'))
dfRoutePrefix

,routeFrom,routeTo,prefix
0,1,14,Hwy
1,15,15,I-
2,16,69,Hwy
3,70,70,I-
4,71,79,Hwy
5,80,80,I-
6,81,83,Hwy
7,84,84,I-
8,85,214,Hwy
9,215,215,I-


In [24]:
dfRoutes = sdfCombinedForecasts.groupby(['ROUTE','CO_FIPS'],as_index=False).agg(numSeg=('SEGID','count'))
dfRoutes['CO_FIPS'  ] = dfRoutes['CO_FIPS'].astype(int)
dfRoutes['ROUTE_numeric'] = pd.to_numeric(dfRoutes['ROUTE'], errors='coerce')

dfRoutes

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric
0,0006,7,22,6
1,0006,15,5,6
2,0006,23,6,6
3,0006,27,16,6
4,0006,49,36,6
...,...,...,...,...
1518,UDOT,53,2,<NA>
1519,WFRC,3,24,<NA>
1520,WFRC,11,47,<NA>
1521,WFRC,35,300,<NA>


In [25]:
# Step 1: Add a dummy key to both DataFrames
dfRoutes['key'] = 1
dfRoutePrefix['key'] = 1

# Step 2: Merge on the dummy key
merged_df = pd.merge(dfRoutes, dfRoutePrefix, on='key')

# Step 3: Filter rows where ROUTE_numeric falls within the routeFrom and routeTo range
# Modify the filter to preserve NaN values (Step 3)
# We use | (OR) to add a condition that explicitly checks for NaNs in ROUTE_numeric
dfRoutePrefixWithPrefix = merged_df[((merged_df['ROUTE_numeric'] >= merged_df['routeFrom']) & 
                                     (merged_df['ROUTE_numeric'] <= merged_df['routeTo'])) |
                                     (merged_df['ROUTE_numeric'].isna())].copy()

# Fill specific columns with '' where ROUTE_numeric is NaN
# Adjust 'your_column_name_here' to the actual column(s) you want to fill for NaN ROUTE_numeric rows
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'prefix'   ] = dfRoutePrefixWithPrefix['ROUTE']
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeFrom'] = 0
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeTo'  ] = 0

dfRoutePrefixWithPrefix['ROUTE_numeric'].fillna(0, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop_duplicates()

# Step 4: Drop the dummy key column
dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop(columns=['key'])
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix
0,0006,7,22,6,1,14,Hwy
12,0006,15,5,6,1,14,Hwy
24,0006,23,6,6,1,14,Hwy
36,0006,27,16,6,1,14,Hwy
48,0006,49,36,6,1,14,Hwy
...,...,...,...,...,...,...,...
18216,UDOT,53,2,0,0,0,UDOT
18228,WFRC,3,24,0,0,0,WFRC
18240,WFRC,11,47,0,0,0,WFRC
18252,WFRC,35,300,0,0,0,WFRC


In [26]:
dfRoutePrefixWithPrefix['ROUTE_text'] = dfRoutePrefixWithPrefix['ROUTE_numeric'].apply(lambda x: str(int(x)) if pd.notnull(x) else np.nan)
dfRoutePrefixWithPrefix.loc[(dfRoutePrefixWithPrefix['ROUTE_text']=="0"), 'ROUTE_text'] = ''
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix,ROUTE_text
0,0006,7,22,6,1,14,Hwy,6
12,0006,15,5,6,1,14,Hwy,6
24,0006,23,6,6,1,14,Hwy,6
36,0006,27,16,6,1,14,Hwy,6
48,0006,49,36,6,1,14,Hwy,6
...,...,...,...,...,...,...,...,...
18216,UDOT,53,2,0,0,0,UDOT,
18228,WFRC,3,24,0,0,0,WFRC,
18240,WFRC,11,47,0,0,0,WFRC,
18252,WFRC,35,300,0,0,0,WFRC,


In [27]:
dfRoutePrefixWithPrefix.fillna('',inplace=True)
dfRoutePrefixWithPrefix['label']=dfRoutePrefixWithPrefix['prefix'] + dfRoutePrefixWithPrefix['ROUTE_text']

dfRoutePrefixWithPrefix.rename(columns={'ROUTE':'R','CO_FIPS':'C'}, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix[['R','C','label']]

dfRoutePrefixWithPrefix

,R,C,label
0,0006,7,Hwy6
12,0006,15,Hwy6
24,0006,23,Hwy6
36,0006,27,Hwy6
48,0006,49,Hwy6
...,...,...,...
18216,UDOT,53,UDOT
18228,WFRC,3,WFRC
18240,WFRC,11,WFRC
18252,WFRC,35,WFRC


In [28]:
# export to JSON
dfRoutePrefixWithPrefix.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/routes.json', orient='records', indent=4)

# Create Forecasts json

In [29]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^F\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 5]

# Display the list of columns
print(columns_matching_pattern)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justneededcols = sdfCombinedForecasts[['SEGID','CO_FIPS'] + columns_matching_pattern]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justneededcols

['F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050']


,SEGID,CO_FIPS,F2019,F2023,F2028,F2032,F2042,F2050
0,0007_000.0,53,3032,5800,5700,5900,9100,11000
1,0007_000.1,53,3032,3400,6800,16000,28500,37500
2,0007_000.3,53,4357,5500,13500,22500,42000,53000
3,0007_001.5,53,4357,4800,7300,9700,23000,28500
4,0007_002.4,53,4357,4800,7300,9100,13500,16500
...,...,...,...,...,...,...,...,...
8932,UDOT_7029,45,0,9500,12000,13500,20000,24500
8933,UDOT_7035,45,-2000,9500,12000,13500,20000,24500
8934,UDOT_7040,29,0,2000,2200,2500,3100,3700
8935,UDOT_7060,45,0,0,0,0,0,5300


In [30]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justneededcols, id_vars=[col for col in sdfCombinedForecasts_justneededcols.columns if col not in columns_matching_pattern], 
                    value_vars=columns_matching_pattern, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[1:]).astype(int)

melted_df['F'] = melted_df['Value'].astype(int)

melted_df['CO_FIPS'] = melted_df['CO_FIPS'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justneededcols_melted = melted_df[['S','Y','F','CO_FIPS']]
sdfCombinedForecasts_justneededcols_melted

,S,Y,F,CO_FIPS
0,0007_000.0,2019,3032,53
1,0007_000.1,2019,3032,53
2,0007_000.3,2019,4357,53
3,0007_001.5,2019,4357,53
4,0007_002.4,2019,4357,53
...,...,...,...,...
53137,UDOT_7029,2050,24500,45
53138,UDOT_7035,2050,24500,45
53139,UDOT_7040,2050,3700,29
53140,UDOT_7060,2050,5300,45


In [31]:
# export to JSON
sdfCombinedForecasts_justneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/forecasts.json', orient='records', indent=4)

# Create Observed

In [32]:
sdfCombinedForecasts.columns

Index(['FID', 'SEGID', 'ROUTE', 'BMP', 'EMP', 'DISTANCE', 'CO_FIPS',
       'SUBAREAID', 'PLANAREA', 'F2019',
       ...
       'FAC_JUL', 'FAC_AUG', 'FAC_SEP', 'FAC_OCT', 'FAC_NOV', 'FAC_DEC',
       'FAC_MAXMO', 'FAC_MAX', 'FACMANADJ', 'SHAPE'],
      dtype='object', length=121)

In [33]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^AADT\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern_AADT = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 8]

# Display the list of columns
print(columns_matching_pattern_AADT)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justAADTneededcols = sdfCombinedForecasts[['SEGID'] + columns_matching_pattern_AADT]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justAADTneededcols

['AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981']


,SEGID,AADT2022,AADT2021,AADT2020,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,...,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981
0,0007_000.0,4837,4273,3011,3032,2978,2883,1457,1380,1320,...,0,0,0,0,0,0,0,0,0,0
1,0007_000.1,4837,4273,3011,3032,2978,2883,1457,1380,1320,...,0,0,0,0,0,0,0,0,0,0
2,0007_000.3,8545,7549,4327,4357,3612,2883,2210,1700,1445,...,0,0,0,0,0,0,0,0,0,0
3,0007_001.5,8545,7549,4327,4357,3612,2883,2210,1700,1445,...,0,0,0,0,0,0,0,0,0,0
4,0007_002.4,8545,7549,4327,4357,3612,2883,2210,1700,1445,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8932,UDOT_7029,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8933,UDOT_7035,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8934,UDOT_7040,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8935,UDOT_7060,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
sdfCombinedForecasts_justAADTneededcols.columns

Index(['SEGID', 'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018',
       'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012',
       'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006',
       'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000',
       'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994',
       'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988',
       'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982',
       'AADT1981'],
      dtype='object')

In [35]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justAADTneededcols, id_vars=[col for col in sdfCombinedForecasts_justAADTneededcols.columns if col not in columns_matching_pattern_AADT], 
                    value_vars=columns_matching_pattern_AADT, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[4:]).astype(int)

melted_df = melted_df[melted_df['Year']>=2000]

melted_df['O'] = melted_df['Value'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justAADTneededcols_melted = melted_df[['S','Y','O']]
sdfCombinedForecasts_justAADTneededcols_melted

,S,Y,O
0,0007_000.0,2022,4837
1,0007_000.1,2022,4837
2,0007_000.3,2022,8545
3,0007_001.5,2022,8545
4,0007_002.4,2022,8545
...,...,...,...
203706,UDOT_7029,2000,0
203707,UDOT_7035,2000,0
203708,UDOT_7040,2000,0
203709,UDOT_7060,2000,0


In [36]:
sdfCombinedForecasts_justAADTneededcols_melted[sdfCombinedForecasts_justAADTneededcols_melted['S']=='0015_000.0']

,S,Y,O
54,0015_000.0,2022,30108
8911,0015_000.0,2021,29259
17768,0015_000.0,2020,23546
26625,0015_000.0,2019,24681
35482,0015_000.0,2018,24316
44339,0015_000.0,2017,23224
53196,0015_000.0,2016,21992
62053,0015_000.0,2015,20195
70910,0015_000.0,2014,18910
79767,0015_000.0,2013,18630


In [37]:
# export to JSON
sdfCombinedForecasts_justAADTneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/observed.json', orient='records', indent=0)

# create segments json

In [38]:
dfSegments = pd.merge(sdfCombinedForecasts[['SEGID','ROUTE','BMP']],dfRoutePrefixWithPrefix, left_on='ROUTE', right_on='R')
dfSegments.rename(columns={'SEGID':'S','label':'N'},inplace=True)

# set label
dfSegments['L'] = dfSegments['N'] + ' ~MP' + dfSegments['BMP'].round(1).astype(str)
dfSegments.loc[(dfSegments['R'].isin(['WFRC','MAG','CACHE','DIXIE','UDOT','UTA'])), 'L'] = dfSegments['S']


dfSegments.drop(columns=['ROUTE','BMP'],inplace=True)
dfSegments

,S,R,C,N,L
0,0007_000.0,0007,53,Hwy7,Hwy7 ~MP0.0
1,0007_000.1,0007,53,Hwy7,Hwy7 ~MP0.1
2,0007_000.3,0007,53,Hwy7,Hwy7 ~MP0.3
3,0007_001.5,0007,53,Hwy7,Hwy7 ~MP1.5
4,0007_002.4,0007,53,Hwy7,Hwy7 ~MP2.4
...,...,...,...,...,...
18878,3464_000.6,3464,57,FA3464,FA3464 ~MP0.6
18879,3468_000.0,3468,57,FA3468,FA3468 ~MP0.0
18880,3469_000.0,3469,57,FA3469,FA3469 ~MP0.0
18881,3478_000.0,3478,57,FA3478,FA3478 ~MP0.0


In [39]:
# export to JSON
dfSegments.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/segments.json', orient='records', indent=0)

In [40]:
dfSegments[dfSegments['C']==21]

,S,R,C,N,L
81,0014_000.0,0014,21,Hwy14,Hwy14 ~MP0.0
83,0014_000.3,0014,21,Hwy14,Hwy14 ~MP0.3
85,0014_000.4,0014,21,Hwy14,Hwy14 ~MP0.4
87,0014_000.5,0014,21,Hwy14,Hwy14 ~MP0.5
89,0014_001.0,0014,21,Hwy14,Hwy14 ~MP1.0
...,...,...,...,...,...
7224,UDOT_7029,UDOT,21,UDOT,UDOT_7029
7229,UDOT_7035,UDOT,21,UDOT,UDOT_7035
7234,UDOT_7040,UDOT,21,UDOT,UDOT_7040
7239,UDOT_7060,UDOT,21,UDOT,UDOT_7060
